**This document will serve as a step by step report, on how the data was prepared for analysis, and later how it was visualized**

In [1]:
# Relevantni importi
% matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from Orange.data.filter import SameValue
from Orange.data import Table
import numpy as np
from csv import DictReader

F:\anacondapy\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Odprem podatke za branje
crash1_reader = DictReader(open('data/Crash_Data_1.csv', 'rt', encoding='utf-8'))
crash2_reader = DictReader(open('data/Crash_Data_2.csv', 'rt', encoding='utf-8'))
liquor_reader = DictReader(open('data/Liquor_Data.csv', 'rt', encoding='utf-8'))

# crash1_o = Table("data/Crash_Data_1.csv")
# crash2_o = Table("data/Crash_Data_2.csv")
# liquor_o = Table("data/Liquor_Data.csv")

In [3]:
# Pogledam imena polj v podatkih
# print(crash1_o.domain)
# print(crash2_o.domain)
# print(liquor_o.domain)
print(crash1_reader.fieldnames)
print("--------------")
print(crash2_reader.fieldnames)
print("--------------")
print(liquor_reader.fieldnames)

['\ufeffX', 'Y', 'OBJECTID', 'CRASH_KEY', 'CASENUMBER', 'LECASENUM', 'CRASH_DATE', 'CRASH_MONTH', 'CRASH_DAY', 'TIMESTR', 'DISTRICT', 'COUNTY_NUMBER', 'CITYNAME', 'SYSTEMSTR', 'LITERAL', 'FRSTHARM', 'LOCFSTHRM', 'CRCOMNNR', 'MAJCSE', 'DRUGALC', 'ECNTCRC', 'LIGHT', 'CSRFCND', 'WEATHER', 'RCNTCRC', 'RDTYP', 'PAVED', 'WZRELATED', 'CSEV', 'FATALITIES', 'INJURIES', 'MAJINJURY', 'MININJURY', 'POSSINJURY', 'UNKINJURY', 'PROPDMG', 'VEHICLES', 'TOCCUPANTS', 'REPORT', 'XCOORD', 'YCOORD']
--------------
['\ufeffX', 'Y', 'OBJECTID', 'VEH_CRASH_KEY', 'VEH_UNITKEY', 'CASENUMBER', 'DRIVERAGE', 'DRIVERGEN', 'DL_STATE', 'CHARGED', 'ALCRESULT', 'DRUGTEST', 'DRUGRESULT', 'DRIVERCOND', 'VISIONOBS', 'DCONTCIRC1', 'DCONTCIRC2', 'VCONFIG', 'CARGOBODY', 'VYEAR', 'MAKE', 'MODEL', 'STYLE', 'VLP_STATE', 'OCCUPANTS', 'VACTION', 'SEQEVENTS1', 'SEQEVENTS2', 'SEQEVENTS3', 'SEQEVENTS4', 'MOSTHARM', 'SPEEDLIMIT', 'TRAFCONT', 'FIXOBJSTR', 'MOSTDAMAGE', 'DAMAGE', 'CSEVERITY', 'MAJORCAUSE', 'CSURFCOND', 'DRUGALCREL', 'RO

In [4]:
# Pregledam kako vnosi podatkov zgledajo
for crash in crash1_reader:
    print(crash)
    break
print("------------------")
for crash in crash2_reader:
    print(crash)
    break
print("------------------")
for liquor in liquor_reader:
    print(liquor)
    break

OrderedDict([('\ufeffX', '-93.511904910700665'), ('Y', '41.648063161780378'), ('OBJECTID', '1001'), ('CRASH_KEY', '2016902721'), ('CASENUMBER', '2016902721'), ('LECASENUM', '16-452'), ('CRASH_DATE', '2016-01-20T00:00:00.000Z'), ('CRASH_MONTH', '1'), ('CRASH_DAY', '4'), ('TIMESTR', '20:02'), ('DISTRICT', '1'), ('COUNTY_NUMBER', '77'), ('CITYNAME', '132'), ('SYSTEMSTR', 'US 6'), ('LITERAL', 'US 6/HUBBELL AVE/NE RAMP/SPEC CASE/FREDERICK M HUBBELL AVE'), ('FRSTHARM', '33'), ('LOCFSTHRM', '1'), ('CRCOMNNR', '4'), ('MAJCSE', '9'), ('DRUGALC', '8'), ('ECNTCRC', '1'), ('LIGHT', '5'), ('CSRFCND', '1'), ('WEATHER', '1'), ('RCNTCRC', '1'), ('RDTYP', '22'), ('PAVED', '1'), ('WZRELATED', ''), ('CSEV', '5'), ('FATALITIES', '0'), ('INJURIES', '0'), ('MAJINJURY', '0'), ('MININJURY', '0'), ('POSSINJURY', '0'), ('UNKINJURY', '0'), ('PROPDMG', '6000'), ('VEHICLES', '2'), ('TOCCUPANTS', '2'), ('REPORT', '7'), ('XCOORD', '457373'), ('YCOORD', '4610828')])
------------------
OrderedDict([('\ufeffX', '-92.80

**Ker so nefiltrirani podatki različnih velikosti, in imajo različno število vnosov, jih sedaj moram filtrirati po neki kriterijski funkciji**
Crash_Data_1.csv je velik 73 Mb
Crash_Data_2.csv je velik 123 Mb
Liquor_Data.csv je velik 3,47 Gb

In [15]:
# Prvo pregledamo število vnosov v posameznih datotekah,
# Vemo da imajo vsi podatke o datumu, tako da po intuiciji bi lahko
# filtrirali po tem.
crash1_row_count = sum(1 for crash in crash1_reader)
crash2_row_count = sum(1 for crash in crash2_reader)
liquor_row_count = sum(1 for liquor in liquor_reader)

print("Crash1 ima "+ str(crash1_row_count) +" vrstic.") # 73Mb z 350 066 vrstic
print("Crash2 ima "+ str(crash2_row_count) +" vrstic.") # 123Mb z 602 387 vrstic
print("Liquor ima "+ str(liquor_row_count) +" vrstic.") # 3,47 Gb z 12 591 076 vrstic
# Ker je čas branja pre velik, so vrstice zapisane v komentarjih zgoraj

KeyboardInterrupt: 

** Recimo da bi bilo smiselno, in časovno učinkovito, imeti enako število vnosov za vse tri. **

** Ena rešitev tega problema, ki jo bom tudi uporabil, je da zmanjšam število vrstic Crash2 in Liquor podatkov na 350 066, kar je minimum od treh vrednosti, zato da niso datoteke pre velike. **

** Ker bi metoda uporabe števca za koliko vrstic je prineslo težave (mogoče spustimo vrednosti za določen datum) bom raje z uporabu naključnosti se odločil če uporabim ta vnos, in če ga, prištejem števec, ko bo števec 350 066 se pisanje fila konča **

In [5]:
import random
import csv

In [6]:
with open('Crash_Data_2_filt.csv', 'w',encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=crash2_reader.fieldnames)
    writer.writeheader()
    count = 1
    for crash in crash2_reader:
        coin = random.randint(1, 3)
        if coin > 1: 
            writer.writerow(crash) 
            count += 1
        if count == 350066: break
print(count)
print("Writing complete")

350066
Writing complete


In [7]:

with open('Liquor_Data_filt.csv', 'w',encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=liquor_reader.fieldnames)
    writer.writeheader()
    count = 1
    for liquor in liquor_reader:
        coin = random.randint(1, 33)
        if coin == 1: 
            writer.writerow(liquor) 
            count += 1
        if count == 350066: break
print(count)
print("Writing complete")

350066
Writing complete


** Deliv sem 350066 z številom vrstic od zgoraj, da sem dobil približno katera verjetnost mora biti, da dokaj uravnoteženo filtriram podatke (2/3 je za crash2, 1/33 za liquor (približno))**

In [11]:
# Filtrirani podatki naloženi
crash1_filt_reader = DictReader(open('data/Crash_Data_1_filt.csv', 'rt', encoding='utf-8'))
crash2_filt_reader = DictReader(open('data/Crash_Data_2_filt.csv', 'rt', encoding='utf-8'))
liquor_filt_reader = DictReader(open('data/Liquor_Data_filt.csv', 'rt', encoding='utf-8'))
